# PROGRES 2024 - Mini-Projet 2
# API Web

Fabien Mathieu - fabien.mathieu@normalesup.org

Sébastien Tixeuil - Sebastien.Tixeuil@lip6.fr

The purpose of this mini-project is to work with the *Internet Movie DataBase* (IMDB) and the Python package bottle. It will involve:

- Retrieve and manipulate datasets
- Build an API to perform various tasks on the data
- Build a website that will use the API above

# Rules

1. Cite your sources
2. One file to rule them all
3. Explain
4. Execute your code


https://github.com/balouf/progres/blob/main/rules.ipynb

# The IMDB dataset

[IMDB](https://www.imdb.com) allows to retrieve a part of its dataset for any non-commercial purpose. The available data and the formatting convention is described here: https://developer.imdb.com/non-commercial-datasets/

We are especially interested in the data from the following files:
- https://datasets.imdbws.com/title.principals.tsv.gz
- https://datasets.imdbws.com/name.basics.tsv.gz
- https://datasets.imdbws.com/title.basics.tsv.gz

**Important notes**:
- If you see *Your answer here*, that means something is expected from you.
- To help you, the start and/or the end of a possible solution is sometimes given.
- The content of IMDB is refreshed regularly. That means that some of the results you will compute, like the number of movies, will vary with time. This should not surprise you.

## Exercise 1: Download

Write a `download_imdb` function inspired by the `download` function seen in course, with the following modifications:
- `download_imdb` will have one single argument, the name of the file to retrieve. Location is assumed to be https://datasets.imdbws.com/
- If the file already exists, print a message telling that it exists and do nothing. You can use the `pathlib` module for that.

Your answer here.

<span style="color:blue"><b>Solution</b></span>La fonction `download_imdb` a pour objectif de télécharger un fichier depuis `https://datasets.imdbws.com/`, en vérifiant d'abord s'il existe déjà dans le dossier courant. Pour cela, nous nous sommes inspirées de la fonction `download` vue en cours ( Web Services),  en reprenant les bases, mais avec quelques modifications afin d'adapter la solution à l'énoncé. 
- D'abord, nous créons une session `requests` pour gérer la requête HTTP, en définissant `s.verify=True` pour s'assurer que la connexion HTTPS est sécurisée, contrairement au cours où `verify` est `False`.
- Ensuite, l'URL complète du fichier est construite via `urljoin`, qui combine `base_url` et le nom du fichier donné en paramètre. Le fichier de destination est ensuite défini via `Path`, qui permet une manipulation compatible sur toutes les plateformes. ( Nous avons demandé à chatgpt s'il y avait un moyen simple de combiner une url et il nous a proposé plusieurs alternatives, parmi lesquelles figurait `urljoin`)
- Nous vérifions ensuite si le fichier est déja présent avec  `Path(file).exists()`  et si c'est le cas, un message est affiché et la fonction s'arrête car il n'y a pas  besoin de télécharger à nouveau .
- Enfin, pour télécharger le fichier, nous utilisons le streaming (`iter_content(chunk_size=8192)`) pour lire et écrire le fichier par blocs, une approche inspirée de la fonction de cours, et qui est efficace pour les fichiers volumineux (comme c'est le cas dans le cadre de ce projet).


In [24]:
from pathlib import Path
from requests import Session
from urllib.parse import urljoin

base_url = "https://datasets.imdbws.com/"

def download_imdb(file):
    s = Session()
    s.verify = True
    source_url = urljoin(base_url, file)
    r = s.get(source_url, stream=True) 
    dest = Path(file)  # Chemin de destination dans le dossier courant 
    if dest.exists():
        print(f"{file} already exists.")
    else:
        with open(dest, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
          

In [26]:
files = ['title.principals.tsv.gz', 'name.basics.tsv.gz', 'title.basics.tsv.gz']
for file in files:
    download_imdb(file)

title.principals.tsv.gz already exists.
name.basics.tsv.gz already exists.
title.basics.tsv.gz already exists.


## Exercise 2: Explore

- What is the size of the different files you retrieved? You can use Python or a file explorer, as you prefer.

Your answer here.

<span style="color:blue"><b>Solution :</b></span>
Le but de cet excercice est de lire les premières lignes d'un fichier TSV compressé (gzip) sans le décompresser entièrement, puis d'analyser le nombre d'entrées pour les films et les personnes dans la base de données pour comprendre comment on peut gérer de gros fichiers compressés, en gérant efficacement la mémoire et en extrayant les informations dont on a besoin uniquement.
Pour la première question de l'excercice, on souhaite calculer la taille des différents fichiers qui figurent dans la liste files, pour cela :
- On utilise `Path` pour obtenir le chemin de chaque fichier dans le répertoire courantet on calcule la taille avec la fonction `st_size` qui permet de récuperer la taille en octets. Pour mieux exprimer la taille, nous avons décider de la convertir en MO en divisant par 1024^2.

In [28]:
files = ['title.principals.tsv.gz', 'name.basics.tsv.gz', 'title.basics.tsv.gz']
for file in files:
    dest = Path(file)  
    size = dest.stat().st_size 
    size_mb = size / (1024**2)
    print(f"{file}: {size} octets, {size_mb:.2f} Mo")

title.principals.tsv.gz: 689764062 octets, 657.81 Mo
name.basics.tsv.gz: 275334300 octets, 262.58 Mo
title.basics.tsv.gz: 197333940 octets, 188.19 Mo


As explained in https://developer.imdb.com/non-commercial-datasets/:
- the data is stored as `tsv`, which means each text line represents a row.
- A [gzip compression](https://docs.python.org/3/library/gzip.html) is used to reduce the size of the data on the hard drive.

Large compressed files should not be uncompressed on your hard drive or fully loaded in memory.

The Python [gzip module](https://docs.python.org/3/library/gzip.html) is designed so you can open a compressed file as if it was already uncompressed. For example, the following code reads 666 lines from `title.basics` and print the last line read.

In [19]:
import gzip
with gzip.open('title.basics.tsv.gz', 'rt', encoding='utf8') as f:
    for _ in range(666):
        l = f.readline()
print(l)

tt0000671	short	Desdemona	Desdemona	0	1908	\N	\N	Drama,Short



- Write a function that read the 4 first lines of a compressed tsv file. Each line read should be converted into a list of elements and printed.

Your answer here.

<span style="color:blue"><b>Solution :</b></span>
Dans cette question, on cherche à lire uniquement les 4 premières lignes du fichier tsv compressé en les convertissant en liste. Pour cela :
- Nous ouvrons le fichier en mode lecture texte (rt) et spécifions l'encodage utf-8 afin d'assurer une bonne interprétation des caractères spéciaux (comme les accents).
- Ensuite, puisqu'il nous a été demandé de lire ligne par ligne, nous utilisons la fonction readline() pour récupérer chaque ligne du fichier.
- Après chaque lecture, nous utilisons strip() pour enlever les espaces en plus et les sauts de ligne à la fin de chaque ligne, ce qui nous permet de ne pas interférer avec le découpage des éléments.
- Puis, comme chaque valeur dans une ligne du fichier TSV est séparée par une tabulation, nous procédons à un découpage de la ligne avec split('\t') pour obtenir les différents éléments sous forme de liste.

In [23]:

def explore(name):
    with gzip.open(name, 'rt', encoding='utf8') as f:
        for _ in range(4):
            l = f.readline().strip()
            liste = l.split('\t')
            print(liste)

In [24]:
for file in files:
    print(f"First lines of {file}:")
    explore(file)

First lines of title.principals.tsv.gz:
['tconst', 'ordering', 'nconst', 'category', 'job', 'characters']
['tt0000001', '1', 'nm1588970', 'self', '\\N', '["Self"]']
['tt0000001', '2', 'nm0005690', 'director', '\\N', '\\N']
['tt0000001', '3', 'nm0005690', 'producer', 'producer', '\\N']
First lines of name.basics.tsv.gz:
['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles']
['nm0000001', 'Fred Astaire', '1899', '1987', 'actor,miscellaneous,producer', 'tt0050419,tt0072308,tt0053137,tt0027125']
['nm0000002', 'Lauren Bacall', '1924', '2014', 'actress,soundtrack,archive_footage', 'tt0037382,tt0075213,tt0117057,tt0038355']
['nm0000003', 'Brigitte Bardot', '1934', '\\N', 'actress,music_department,producer', 'tt0057345,tt0049189,tt0056404,tt0054452']
First lines of title.basics.tsv.gz:
['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']
['tt0000001', 'short', 'Carmencita', 'Carmencita', '0',

- How many movie entries are present in the retrieved database?
- How many people entries?

Your answer here.

<span style="color:blue"><b>Solution :</b></span>
On cherche à présent à compter le nombre de films et de personnes présents dans la base de données. Pour cela :
- On intialise deux compteurs (`count_movies` et `count_people`) à zéro pour stocker le nombre de films et de personnes trouvés dans les fichiers.
- Pour énumérer le nombre de films, on ouvre le premier fichier (`file1`) contenant des informations sur les titres. Pour chaque ligne du fichier, on la découpe en éléments à l'aide de `split('\t')`. Ensuite, si le deuxième élément de la ligne (l'index 1) correspond à 'movie', cela indique qu'il s'agit d'un film, et on incrémente le compteur `count_movies`.
- Pour ce qui en est des personnes, on fait un traitement similaire en utilisant le deuxième fichier (`file2`), qui contient des informations sur les noms. On lit chaque ligne et on la découpe en éléments. Si la ligne n'est pas vide et que le premier élément existe, cela signifie qu'il s'agit d'une entrée valide représentant une personne, et on incrémente le compteur `count_people` de 1.

In [28]:
def movies_entries(file1,file2):
    count_movies = 0
    count_people = 0 
    with gzip.open(file1, 'rt', encoding='utf8') as f1:
        for l in f1 :
            liste = l.split('\t')
            if liste[1] == 'movie' :
                 count_movies += 1 
        print('number of movies' , count_movies)
    with gzip.open(file2, 'rt', encoding='utf8') as f2:
        for k in f2 :
            liste2 = k.split('\t')
            if len(liste2) > 0 and liste2[0] != '':
                 count_people += 1 
        print('number of people' , count_people)
        

file_1 = "title.basics.tsv.gz"
file_2 = "name.basics.tsv.gz"
movies_entries(file_1, file_2)



number of movies 697071
number of people 13944219


## Exercise 3: Extract

We want to study the relations between actors and movies. In particular, we focus on:
- Actual movies (e.g. not TV shows or short movies), where the movie year is known and at least one actor/actress is credited.
- Actors that are credited in at least one actual movie.

To start with, build a [Python set](https://docs.python.org/3/tutorial/datastructures.html#sets) that contains all movie ids (`tconst`) such that:
- The type of movie (`titleType`) is `movie`;
- The year (`startYear`) exists, i.e. is an integer.

How many movies have you referenced in the set?

Your answer here.

<span style="color:blue"><b>Solution :</b></span>
Dans cette partie du code, l'idée était de créer un ensemble, `true_movies`, qui va contenir les identifiants des films valides.
- Pour ça, on ouvre d'abord le fichier `title.basics.tsv.gz` en mode lecture texte, en spécifiant l'encodage UTF-8 pour bien gérer les caractères spéciaux.
- Ensuite, on parcourt chaque ligne du fichier et on découpe les informations avec `split('\t')` pour récupérer les différents champs de chaque film.
-  Ce qu'on veut, c'est filtrer uniquement les films, donc on vérifie que la deuxième colonne de chaque ligne est bien `'movie'`, ce qui nous permet d'ignorer les séries ou autres types.
- Ensuite, on s'assure que l'année de sortie, qui est dans la cinquième colonne, est un nombre valide en utilisant `isdigit()`. Si tout cela est vérifié, on récupère l'identifiant du film, qu'on ajoute à l'ensemble `true_movies` pour qu'au final, cet ensemble contienne uniquement les films qui ont une année de sortie valide et qui sont bien des films.

In [34]:
true_movies = set()
with gzip.open("title.basics.tsv.gz", 'rt', encoding='utf8') as f:
        for l in f: 
            liste = l.split('\t') 
            if liste[1] == 'movie' and liste[5].isdigit(): 
                true_movies.add(liste[0]) 

     


In [35]:
len(true_movies)

595755

Now we want to build two lists, `movies` and `actors`:

- Each element of `movies` should represent a movie, each element of `actors` an actor or actress;
- A movie is represented by a list of three elements:
  - The original name of the movie (`str`),
  - The principal actors of the movie, stored as a list whose elements are integers that represent the index (position) of the actors in the list `actors`,
  - The movie year, `startYear` (`int`);
- An actor/actress is represented by a list of two elements:
  - The name of the person (`str`),
  - The movies the person acted in, stored as a list whose elements are integers that represent the index (position) of the movies in the list `movies`.
  

Build these two lists.

A possible way to do this:
- Initiate `movies` and `actors` as empty lists;
- Create two auxiliary dictionary that will associate to each movie id (`tconst`) and person id (`nconst`) their position in the list;
- Read the file `title.principals.tsv.gz` line by line:
  - Ignore any line where the movie is not in the set `true_movies` or the `category` of the relation is not `actor` or `actress`,
  - If the movie id `tconst` is not in the movie auxiliary index, append an empty movie to `movies` (`["", [], 0]`) and update the movie auxiliary index with an entry for `tconst`,
  - If the actor id `nconst` is not in the actor auxiliary index, append an empty actor to `actors` (`["", []]`) and update the actor auxiliary index with an entry for `nconst`,
  - Append the movie index (not `tconst`!) to the movies of the corresponding actor in `actors`,
  - Append the actor index (not `nconst`!) to the actors of the corresponding movie in `movies`;
- There can be a few undesired duplicates, e.g. some actors can have multiple entries for the same movies. For each actor, remove possible duplicates in the list of movies, and for each movie, remove possible duplicates in the list of actors;
- Using `title.basics.tsv.gz` and your movie auxiliary index, populate each movie in `movies` with its correct name (`str`) and year (`int`);
- Using `name.basics.tsv.gz` and your actor auxiliary index, populate each actor in `movies` with her correct name.

Your answer here.

<span style="color:blue"><b>Solution :</b></span>
Pour répondre à cette question, on a construit deux listes : une pour les films (movies) et une pour les acteurs/actrices (actors). L idée était de lier chaque film à ses acteurs et chaque acteur à ses films, en utilisant des indices pour faire les correspondances. 
- On a commencé par créer deux dictionnaires, movie_id_to_index et actor_id_to_index, qui nous permettent de retrouver rapidement l indice d un film ou d un acteur dans leurs listes respectives.
- Ensuite, on a ouvert le fichier title.principals.tsv.gz et on a parcouru chaque ligne pour récupérer les informations sur les films et les acteurs. À chaque ligne, on vérifie d abord si le film fait partie de notre liste de "true_movies" et si la catégorie est bien "actor" ou "actress". Si ces conditions sont remplies, on ajoute l acteur à la liste des acteurs du film, et on ajoute le film à la liste des films de l acteur.
- Le problème qui est survenu, c’est qu’un acteur peut parfois apparaître plusieurs fois pour le même film, ce qui entraîne des doublons dans les listes. ChatGPT nous a alors aider à résoudre ce problème en utilisant la fonction set(), ce qui nous a permis de supprimer les duplicatas dans les deux listes, aussi bien pour les films que pour les acteurs.
- Ensuite, pour compléter les informations, on a ouvert le fichier title.basics.tsv.gz et on a utilisé les indices des films stockés dans notre dictionnaire movie_id_to_index pour ajouter le nom du film et son année de sortie à chaque entrée de la liste movies. Si l’année n’était pas valide ou absente, on a défini une valeur par défaut de 0. De même, on a ouvert name.basics.tsv.gz pour récupérer les noms des acteurs en utilisant les indices stockés dans actor_id_to_index.

In [40]:
movie_id_to_index = {}
actor_id_to_index = {}
movies = []
actors = []

def movies_actors():
    with gzip.open("title.principals.tsv.gz", 'rt', encoding='utf8') as f:
        for l in f:
            liste = l.strip().split('\t')
            tconst = liste[0]
            nconst = liste[2]
            category = liste[3]

            if tconst in true_movies and category in ('actor', 'actress'):
                
            
                if tconst not in movie_id_to_index:
                    movie_index = len(movies)
                    movies.append(["", [], 0])  
                    movie_id_to_index[tconst] = movie_index
                else:
                    movie_index = movie_id_to_index[tconst]

               
                if nconst not in actor_id_to_index:
                    actor_index = len(actors)
                    actors.append(["", []])  
                    actor_id_to_index[nconst] = actor_index
                else:
                    actor_index = actor_id_to_index[nconst]

               
                actors[actor_index][1].append(movie_index)
                movies[movie_index][1].append(actor_index)

   
    for actor in actors:
        actor[1] = list(set(actor[1]))  
    for movie in movies:
        movie[1] = list(set(movie[1]))  

  
    with gzip.open("title.basics.tsv.gz", 'rt', encoding='utf8') as f:
        for l in f:
            l = l.strip()
            liste = l.split('\t')
            tconst = liste[0]
            if tconst in movie_id_to_index:
                movie_index = movie_id_to_index[tconst]
                movies[movie_index][0] = liste[2] 
                try:
                    movies[movie_index][2] = int(liste[5])  
                except ValueError:
                    movies[movie_index][2] = 0  

   
    with gzip.open("name.basics.tsv.gz", 'rt', encoding='utf8') as f:
        for l in f:
            l = l.strip()
            liste = l.split('\t')
            nconst = liste[0]
            if nconst in actor_id_to_index:
                actor_index = actor_id_to_index[nconst]
                actors[actor_index][0] = liste[1]  


movies_actors()


print("Total movies:", len(movies))
print("Total actors:", len(actors))


Total movies: 470189
Total actors: 1197712


Manually check that your files are correct. For example, try to get the name and year of the movies Michel Blanc played in, or the actors of the first Harry Potter movie.

Your answer here (if everything went well, you just need to execute the two cells below).

In [43]:
', '.join([f"{movies[i][0]} ({movies[i][2]})" for i in [a for a in actors if a[0]=='Michel Blanc'][0][1]])

"Les petites victoires (2023), The Best Way to Walk (1976), The Favour, the Watch and the Very Big Fish (1991), Gramps Is in the Resistance (1983), Take a Chance on Me (2023), Dream one (1984), Move Along, There is Nothing to See (1983), Uranus (1990), R.A.I.D. Special Unit (2016), Cause toujours... tu m'intéresses! (1979), Out of Whack (1979), You Are So Beautiful (2005), Prospero's Books (1991), The Horse of Pride (1980), Summer Things (2002), Ma femme s'appelle reviens (1982), Separate Bedrooms (1989), Toxic Affair (1993), The Girl on the Train (2009), The Minister (2011), Kiss & Tell (2018), The Witnesses (2007), The New Beaujolais Wine Has Arrived... (1978), Odd Job (2016), Le routard (2025), Madame Edouard (2004), The Day I Saw Your Heart (2011), The Hundred-Foot Journey (2014), A Spot of Bother (2009), Top Dogs (2022), Viens chez moi, j'habite chez une copine (1981), You Won't Have Alsace-Lorraine (1977), A Good Doctor (2019), Marche à l'ombre (1984), I Hate Actors (1986), Santa

In [44]:
', '.join([actors[i][0] for i in [m for m in movies if m[0].startswith('Harry Potter')][0][1]])

'Saunders Triplets, Harry Melling, Daniel Radcliffe, Maggie Smith, Richard Griffiths, Fiona Shaw, Richard Harris, Robbie Coltrane, Rupert Grint, Emma Watson'

When you have successfully reached this point of the project, you can save the two lists `movies` and `actors` as compressed json files using the code below:

In [46]:
import gzip
import json

with gzip.open('movies.json.gz', 'wt', encoding='utf8') as f:
    json.dump(movies, f)
with gzip.open('actors.json.gz', 'wt', encoding='utf8') as f:
    json.dump(actors, f)

After your files have been saved, you do not need to re-execute all of the above each time your restart your notebook. Instead, you just need to reload `movies` and `actors` using the code below:

In [48]:
%%writefile utils.py


import gzip
import json

with gzip.open('movies.json.gz', 'rt', encoding='utf8') as f:
    movies = json.load(f)
with gzip.open('actors.json.gz', 'rt', encoding='utf8') as f:
    actors = json.load(f)  
print("done")

Overwriting utils.py


In [49]:
from utils import movies, actors

print(movies[:5]) 
print("\n\n")
print(actors[:5]) 

done
[['Miss Jerry', [0, 1, 2], 1894], ['Bohemios', [3, 4], 1905], ['The Story of the Kelly Gang', [5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 1906], ['The Prodigal Son', [16, 17, 18, 15], 1907], ['Robbery Under Arms', [19, 20, 21, 22, 23, 24], 1907]]



[['Blanche Bayliss', [0]], ['William Courtenay', [0, 2401, 12771, 1736, 88777, 2378, 11919, 11956, 12725, 1590, 11543, 1144, 2652]], ['Chauncey Depew', [0]], ['Antonio del Pozo', [1]], ['El Mochuelo', [1]]]


**Important remark:** in what follows, you will have to build functions that use the two lists a lot. You should NOT reload the lists each time you call a function. Instead, ensure that the two lists are loaded in memory and use them directly.

## Exercise 4: Explore again (now on the curated dataset)

- How many actors do you have in the new dataset? How many movies?
- In average, in how many movies played an actor?
- In average, how many actors play in a movie?
- What is the name of the actor that played in the most movies? How many movies did he feature in?
- What is the oldest movie in the DB?

Your answer here.

<span style="color:blue"><b>Solution :</b></span>

<span style="color:green"><b>Q1 :</b></span>  on commence par afficher le nombre total de films et d'acteurs dans les bases de données. On utilise len(actors) pour obtenir le nombre d'acteurs et len(movies) pour obtenir le nombre de films.

<span style="color:green"><b>Q2 :</b></span> L'objectif est de savoir combien de films au total ont été joués par tous les acteurs. Pour cela, on parcourt la liste des acteurs à l'aide d'une boucle for. Pour chaque acteur, on accède à la liste des films dans lesquels il a joué, grâce à actor[1] (la deuxième valeur du tuple représente un acteur, où chaque acteur est stocké sous la forme (id_actor, list_of_movies)). Ensuite, on ajoute la taille de cette liste (len(actor[1])) au total global nb_movie_played. Après avoir accumulé le nombre total de films, on calcule la moyenne des films par acteur en divisant ce total par le nombre d'acteurs (nb_actors). Si la division est possible, on affiche la moyenne.

<span style="color:green"><b>Q3 :</b></span> on nous demande combien d'acteurs apparaissent en moyenne dans chaque film. Comme dans la question précédente, on parcourt cette fois la liste des films, et pour chaque film , on compte combien d'acteurs sont associés au film en utilisant len(movie[1]). Le total des acteurs par film est ensuite divisé par le nombre de films (nb_movies) pour obtenir la moyenne. Si la liste de films est vide, la division ne se fait pas pour éviter une erreur de division par zéro.

<span style="color:green"><b>Q4 :</b></span> ici, on cherche l'acteur qui a joué dans le plus grand nombre de films. Pour cela, on initialise une variable max_nb_movies à zéro et une variable id_actor pour stocker l'identifiant de l'acteur. Ensuite, on parcourt chaque acteur et on compare le nombre de films dans lesquels il a joué avec max_nb_movies. Si un acteur a joué dans plus de films que le précédent maximum, on met à jour max_nb_movies et id_actor pour stocker le nouvel acteur avec le plus grand nombre de films. Enfin, après avoir parcouru tous les acteurs, on affiche l'identifiant de l'acteur et le nombre de films où il a joué.

<span style="color:green"><b>Q5 :</b></span> Cette dernière question nous demande de trouver le film le plus ancien. On initialise oldest_year à une valeur élevée (9999) pour garantir que n'importe quel film aura une année plus basse. Ensuite, on parcourt la liste des films et, pour chaque film, on vérifie si son année de sortie (çàd movie[2]) est plus ancienne que l'année que l'on garde en mémoire. Si c'est le cas, on met à jour la variable oldest_year et id_movie avec l'identifiant du film. À la fin de la boucle, on affiche l'identifiant du film le plus ancien et l'année de sa sortie.

In [55]:
def count_actors_movies():
    # Q1
    nb_actors = len(actors)
    nb_movies = len(movies)
    print("Nombre de films : ", nb_movies)
    print("Nombre d'acteurs : ", nb_actors)

    # Q2
    nb_movie_played = 0
    nb_actors = len(actors)  

    for actor in actors:
        nb_movie_played += len(actor[1])  
    if nb_actors: 
        average_number = nb_movie_played / nb_actors
        print("Le nombre moyen de films par acteur est :", average_number)

    # Q3
    nb_actors_per_movie = 0
    for movie in movies :
        nb_actors_per_movie +=  len(movie[1])
    if nb_movies :
        average_actors = nb_actors_per_movie / nb_movies
    else : return 
    print("Le nombre moyen d'acteurs par film : " , average_actors)
    
    
    # Q4
    id_actor = ""
    max_nb_movies = 0
    for actor in actors :
        nb_movies = len(actor[1])
        if nb_movies > max_nb_movies :
            id_actor = actor[0]
            max_nb_movies = nb_movies
        else : 
            continue       
    print("L'acteur qui a joué dans le plus de film : ", id_actor, "avec : " , max_nb_movies , " films") 

    # Q5
    oldest_year = 9999
    id_movie = ""
    for movie in movies :
        if int(movie[2]) > 0 :
            if int(movie[2]) < oldest_year  :
                oldest_year = int(movie[2])
                id_movie = movie[0]
        else : 
            continue
    print("Le film le + ancien enregistré dans la bd est : " ,id_movie , "joué en " , oldest_year)
    
count_actors_movies()



Nombre de films :  470189
Nombre d'acteurs :  1197712
Le nombre moyen de films par acteur est : 3.027677772285825
Le nombre moyen d'acteurs par film :  7.712400757993063
L'acteur qui a joué dans le plus de film :  Brahmanandam avec :  1122  films
Le film le + ancien enregistré dans la bd est :  Miss Jerry joué en  1894


## Exercise 5: Prepare some functions

Write the following functions
- `search_movie(name: str) -> list`: return a list of movies whose name contains `name` (ignoring case). Each movie is described as a dictionary with keys `name`, `year`, and `index` (its position in `movies`)
- `get_movie(i: int) -> dict`: returns the a json of the movie at position `i`, with following keys:
  - `name` (`str`)
  - `year` (`int`)
  - `actors` (list of dictionaries with keys `name` and `index`)
- `search_actor(name: str) -> list`: return a list of actors whose name contains `name` (ignoring case). Each actor is described as a dictionary with keys `name` and `index` (its position in `actor`)
- `get_actor(i: int) -> dict`: returns the a json of the actor at position `i`, with following keys:
  - `name` (`str`)
  - `movies` (list of dictionaries with keys `name`, `year`, and `index`)

Your answer here.

<span style="color:blue"><b>Solution :</b></span>


<span style="color:orange"><b>NOTE:</b></span> Nous avons fait le choix d'écrire toutes ces fonctions dans un fichier utils.py car elles seront utilisées à différents moments du projet. Pour garantir que les modifications apportées à ce fichier sont bien prises en compte, nous avons utilisé la commande %%writefile -a utils.py pour ajouter les fonctions à ce fichier sans écraser son contenu existant.

<span style="color:green"><b>search_movie(name:str):</b></span> elle a pour but de rechercher dans une base de films ceux dont le nom contient un mot spécifique, tout en ignorant la casse.
- Pour y parvenir, on a utilisé une expression régulière générée avec l'aide de ChatGPT. On utilise la fonction re.search() pour trouver une correspondance avec cette expression régulière dans le titre du film.
- L'expression régulière qu'on a choisie est r'\b' + re.escape(name) + r'\b'. Le \b permet de s'assurer qu'on cherche un mot entier. Cela évite par exemple de trouver des correspondances dans des morceaux de mots. Ensuite, on utilise re.escape(name) pour protéger le mot recherché contre d'éventuels caractères spéciaux, comme des parenthèses ou des points, qui pourraient altérer la recherche.
- Enfin, on ajoute re.IGNORECASE pour ne pas tenir compte de la casse. Cela veut dire que peu importe si le mot est en majuscules ou minuscules, on le retrouvera dans les titres.

  
<span style="color:green"><b>get_movie(i:int):</b></span> elle a pour but de récupérer les informations d'un film à partir de son index dans la liste movies, qui est une liste de films où chaque élément contient des détails comme le nom du film, l'année, etc.
- On parcourt cette liste avec enumerate() pour trouver le film correspondant à l'index i.
-  Ensuite, on se concentre sur les acteurs de ce film, en vérifiant dans la liste actors (qui contient des acteurs et les films auxquels ils participent).
- On cherche simplement si l'index du film fait partie des films associés à chaque acteur. Dès qu'on a ces informations, on assemble tout ça dans un dictionnaire et on le retourne.


<span style="color:green"><b>search_actor(name:str):</b></span> a pour but de rechercher des acteurs dont le nom contient un mot spécifique, sans tenir compte de la casse. 
- Pour ça, on parcourt la liste `actors`, qui contient les noms des acteurs et les films auxquels ils participent. Pour chaque acteur, on utilise `re.fullmatch()`, qui permet de nous assurer que le nom de l'acteur correspond exactement au mot qu'on cherche, sans se tromper. C’est un peu comme si on cherchait le mot entier sans qu’il soit un sous-ensemble d’un autre mot.
- Le + avec `re.fullmatch()`, c’est qu’on est sûr qu’on ne trouve que des correspondances exactes pour le mot entier On ajoute aussi `re.IGNORECASE`, ce qui fait que la recherche ignore la casse, donc peu importe si le nom de l’acteur est en majuscules ou minuscules, on le trouvera quand même.
- Si une correspondance est trouvée, nous stockons le nom de l'acteur et son index dans la liste des résultats, et on renvoie cette liste à la fin.
- Cela fonctionne un peu de la même façon que dans `search_movie()`, où l'on utilise aussi une expression régulière pour rechercher un mot dans les titres de films, avec re.IGNORECASE pour gérer la casse. La différence est que dans search_actor, on se concentre sur le nom des acteurs et on utilise fullmatch() pour une correspondance exacte du mot.


<span style="color:green"><b>get_actor(i: int):</b></span> a pour but de récupérer les infos d’un acteur à partir de son index dans la liste `actors`. Chaque acteur dans cette liste est représenté par un tuple, où le premier élément est le nom de l'acteur et le deuxième est une liste d'indices représentant les films auxquels cet acteur a participé dans la liste `movies`.
- On commence par une boucle `for` où on parcourt la liste des acteurs. Pour chaque acteur, on vérifie si son index correspond à l'index donné en argument (`i`). Si c'est le cas, on crée une nouvelle liste `movie_list` pour y stocker les films associés à cet acteur.
-  Ensuite, pour chaque indice de film dans `actor[1]` (la liste des indices de films de l'acteur), on récupère le film correspondant dans la liste `movies`. On crée alors un dictionnaire `movie_info` qui contient le nom du film (`movie[0]`), l'année de sortie (`movie[2]`), et l'index du film dans la liste (`movie_index`).
-  Après avoir collecté toutes les informations sur les films de l'acteur, on construit un dictionnaire `actor_info` qui contient le nom de l'acteur et la liste de ses films sous la clé `'movies'`et qui sera retourné.


In [60]:
%%writefile -a utils.py
import re

def search_movie(name: str) -> list:
    search_result = []
    for i, movie in enumerate(movies):  
        if re.search(r'\b' + re.escape(name) + r'\b', movie[0], re.IGNORECASE):
            result = {'name': movie[0], 'year': movie[2], 'index': i}
            search_result.append(result)

    return search_result  



Appending to utils.py


In [61]:
import importlib
import utils
importlib.reload(utils)
from utils import search_movie

done


In [62]:
movies_found = search_movie("gendarme")
print(movies_found)

[{'name': 'The Gendarme of Saint-Tropez', 'year': 1964, 'index': 41003}, {'name': 'The Gendarme in New York', 'year': 1965, 'index': 42674}, {'name': 'The Gendarme Gets Married', 'year': 1968, 'index': 44456}, {'name': 'The Gendarme Takes Off', 'year': 1970, 'index': 46404}, {'name': 'The Gendarme and the Extra-Terrestrials', 'year': 1979, 'index': 55244}, {'name': 'The Gendarme and the Gendarmettes', 'year': 1982, 'index': 58338}, {'name': 'The Gendarme of Champignol', 'year': 1959, 'index': 92831}, {'name': 'El gendarme de la esquina', 'year': 1951, 'index': 116185}, {'name': "Hainburg - Je t'aime, gendarme", 'year': 2001, 'index': 145905}, {'name': 'Le gendarme de Abobo', 'year': 2019, 'index': 319665}]


In [63]:
%%writefile -a utils.py
from utils import movies,actors

def get_movie(i):
    movie_infos = {}
    list_of_actors = []

    for j, movie in enumerate(movies):
        if j == i:
            for k, actor in enumerate(actors):
                if i in actor[1]: 
                    actor_info = {'name': actor[0], 'index': k}
                    list_of_actors.append(actor_info)
                    
            movie_infos = {
                'name': movie[0],
                'year': movie[2],
                'actors': list_of_actors
            }
            return movie_infos 


Appending to utils.py


In [64]:
import importlib
import utils
importlib.reload(utils)
from utils import get_movie

done


In [65]:
from utils import get_movie
result = get_movie(55030)
print(result)

{'name': 'Brontosaurus', 'year': 1980, 'actors': [{'name': 'Pavla Marsálková', 'index': 69663}, {'name': 'Josef Somr', 'index': 71798}, {'name': 'Daniela Kolárová', 'index': 79803}, {'name': 'Zdenek Sverák', 'index': 93309}, {'name': 'Dana Vávrová', 'index': 100561}, {'name': 'Michal Hofbauer', 'index': 100869}, {'name': 'Tomás Simek', 'index': 107137}, {'name': 'Lukás Bech', 'index': 107138}, {'name': 'Petra Fiserová', 'index': 107139}, {'name': 'Sárka Fiserová', 'index': 107140}]}


In [66]:
%%writefile -a utils.py
from utils import movies,actors

def search_actor(name):
    search_actor = []
    for i, actor in enumerate(actors) :
       if re.fullmatch(r'\b' + name + r'\b', actor[0], re.IGNORECASE):
           result = {'name': actor[0], 'index': i  }
           search_actor.append(result)           
    return search_actor


Appending to utils.py


In [67]:
importlib.reload(utils)
from utils import search_actor

done


In [68]:
actor_info = search_actor('Ramy A. Razek')
print(actor_info)

[{'name': 'Ramy A. Razek', 'index': 548072}]


In [69]:
%%writefile -a utils.py
from utils import movies,actors

def get_actor(i):
    for j, actor in enumerate(actors) :
        movie_list= []
        if i == j :
            name = actor[0]
            for movie_index in actor[1]:
                movie = movies[movie_index]  # Accèder au film à l'indice donné
                movie_info = {'name': movie[0], 'year': movie[2], 'index': movie_index}
                movie_list.append(movie_info)
            
            actor_info = {
                'name': name,
                'movies': movie_list
            }
    
    return actor_info    

Appending to utils.py


In [70]:
importlib.reload(utils)
from utils import get_actor

print(get_actor(548029)) 

done
{'name': 'Ryan Blomberg', 'movies': [{'name': 'Never Go Back', 'year': 2020, 'index': 232536}]}


In [71]:
search_movie('bronzés')

[{'name': "Les P'tits Bronzés au Pyrénéen", 'year': 2013, 'index': 445126}]

In [72]:
get_movie(55030)

{'name': 'Brontosaurus',
 'year': 1980,
 'actors': [{'name': 'Pavla Marsálková', 'index': 69663},
  {'name': 'Josef Somr', 'index': 71798},
  {'name': 'Daniela Kolárová', 'index': 79803},
  {'name': 'Zdenek Sverák', 'index': 93309},
  {'name': 'Dana Vávrová', 'index': 100561},
  {'name': 'Michal Hofbauer', 'index': 100869},
  {'name': 'Tomás Simek', 'index': 107137},
  {'name': 'Lukás Bech', 'index': 107138},
  {'name': 'Petra Fiserová', 'index': 107139},
  {'name': 'Sárka Fiserová', 'index': 107140}]}

In [73]:
get_movie(1)

{'name': 'Bohemios',
 'year': 1905,
 'actors': [{'name': 'Antonio del Pozo', 'index': 3},
  {'name': 'El Mochuelo', 'index': 4}]}

In [74]:
search_actor('Daniel Radcliffe')

[{'name': 'Daniel Radcliffe', 'index': 277317}]

In [75]:
search_actor('jean dujardin')

[{'name': 'Jean Dujardin', 'index': 329574}]

In [76]:
get_actor(277317)

{'name': 'Daniel Radcliffe',
 'movies': [{'name': 'Playmobil: The Movie', 'year': 2019, 'index': 397185},
  {'name': 'Swiss Army Man', 'year': 2016, 'index': 394630},
  {'name': 'Escape from Pretoria', 'year': 2020, 'index': 421517},
  {'name': 'The Lost City', 'year': 2022, 'index': 241935},
  {'name': 'Weird: The Al Yankovic Story', 'year': 2022, 'index': 283921},
  {'name': 'Harry Potter and the Deathly Hallows: Part 2',
   'year': 2011,
   'index': 226975},
  {'name': 'Victor Frankenstein', 'year': 2015, 'index': 298528},
  {'name': 'Harry Potter and the Prisoner of Azkaban',
   'year': 2004,
   'index': 145911},
  {'name': 'National Theatre Live: Rosencrantz & Guildenstern Are Dead',
   'year': 2017,
   'index': 435621},
  {'name': 'Now You See Me 2', 'year': 2016, 'index': 363827},
  {'name': 'Kill Your Darlings', 'year': 2013, 'index': 239158},
  {'name': 'Guns Akimbo', 'year': 2019, 'index': 436286},
  {'name': 'Beast of Burden', 'year': 2018, 'index': 427076},
  {'name': 'Now 

Write a function `movie_path(origin: int, destination: int) -> distance: int, path: list` that computes the collaboration distance between two actors. That distance is the length of the shortest path `(origin, act1, act2, ..., actX, destination)`, where `origin` and `act` played in the same movie, `act1` and `act2` played in the same movie, ... and
`actX` and `destination` played in the same movie.  In addition to the distance, the response should include one shortest path between the two actors, as a list of the form `["origin_name", "movie1_name", "act1_name", "movie2_name", ..., "destination_name"]`, where `movie1` is a movie that featured `origin` and `act1`, and so on...

In particular:
- One actor is by convention at distance 0 from herself. The return path should be `["origin_name"]` then;
- Two distinct actors that play in the same movie are at distance 1;
- If there is no connection between two actors, the function should return `-1, []` by convention.

**Important remarks**: `movie_path` is tricky. You need to try to implement it but you are allowed to fail. If you are stuck for too long, please explain what you did/try and what blocked you in your opinion. Then move on.

Your answer here.

In [79]:
def movie_path(origin, destination):
    ...

In [80]:
search_actor('jean dujardin')

[{'name': 'Jean Dujardin', 'index': 329574}]

In [81]:
search_actor('kiefer sutherland')

[{'name': 'Kiefer Sutherland', 'index': 123745}]

In [82]:
search_actor('kevin bacon')

[{'name': 'Kevin Bacon', 'index': 105317},
 {'name': 'Kevin Bacon', 'index': 1142807}]

In [83]:
search_actor('louis de funès')

[{'name': 'Louis de Funès', 'index': 38481}]

In [84]:
movie_path(105311, 105311)

In [85]:
movie_path(105311, 329504)

In [86]:
movie_path(38476, 123737)

In [87]:
search_movie('gendarme')

[{'name': 'The Gendarme of Saint-Tropez', 'year': 1964, 'index': 41003},
 {'name': 'The Gendarme in New York', 'year': 1965, 'index': 42674},
 {'name': 'The Gendarme Gets Married', 'year': 1968, 'index': 44456},
 {'name': 'The Gendarme Takes Off', 'year': 1970, 'index': 46404},
 {'name': 'The Gendarme and the Extra-Terrestrials',
  'year': 1979,
  'index': 55244},
 {'name': 'The Gendarme and the Gendarmettes', 'year': 1982, 'index': 58338},
 {'name': 'The Gendarme of Champignol', 'year': 1959, 'index': 92831},
 {'name': 'El gendarme de la esquina', 'year': 1951, 'index': 116185},
 {'name': "Hainburg - Je t'aime, gendarme", 'year': 2001, 'index': 145905},
 {'name': 'Le gendarme de Abobo', 'year': 2019, 'index': 319665}]

## Exercise 6. Provide a Web API

Using Python and the Bottle package, build a web server that implements the following API:
- `/movies/{id}` : where `id` is the index of a movie, returns the corresponding movie as a json (cf `get_movie`).
- `/movies` : returns by default the first 100 movies. The value 100 can be modified by sending a URL parameter `limit`.
- `/actors/{id}` : where `id` is the index of an author, returns the json of the actor (cf `get_actor`).
- `/actors` : returns by default the first 100 actors. The value 100 can be modified by sending a URL parameter `limit`.
- `/actors/{id}/costars` : returns the co-stars of one actor (actors that play in a same movie).
- `/search/actors/{searchString}` : where `searchString` is a string to lookup one actor. This route should return the actors whose name contains `searchString` (for example, `/search/actors/w` returns the actors whose name contains `w` or `W`).
- `/search/movies/{searchString}`: where `searchString` is a string, returns the list of movies whose title contains `searchString`. The route should accept a URL parameter `filter` formatted like `key1:value1,key2:value2,...`  to restrain the search to the publications where key `keyi` contains `valuei`. For example, `/search/movies/gendarme?filter=year:1964`
should return the list of movies where the title contains `gendarme` published in 1964.
- `/actors/{id_origin}/distance/{id_destination}` : where `id_origin`
and `id_destination` are two actor indices, returns the collaboration distance between the two actors. In addition to the distance, the response should include one shortest path between the two actors, e.g. the json you return should be a list of two elements, one integer and one list.

The developed API should have the following characteristics:

- All errors should have the same format.
- In absence of error, the API should always return a `json`.
- Each route must be documented with the return format, possible errors, and an explanation of parameters.
- Each route that returns a list should return a maximum of 100 elements and should accept URL parameters `start` and `limit` to display `limit` elements starting from the `start`-th element. For example: `/actors` should return the first 100 authors, `/actors?start=100` displays the next 100, and `/actors?start=200&limit=2` displays the next 2 elements.
- For each route that returns a list, the returned elements should be sortable based on a given field using a URL parameter `order`. For example: `/movies?order=year` displays the first 100 movies sorted by year.

Your answer here.

<span style="color:blue"><b>Solution :</b></span>

- `/movies/<id:int>` - Récupère un film par ID. --> La fonction movie_by_id reçoit un ID de film depuis l'URL et appelle la fonction get_movie(id) pour obtenir les détails du film. Si le film existe, il le retourne au format JSON. Sinon, elle renvoie une réponse d'erreur avec un statut 404 et le message "Film non trouvé".

- `/movies` - Récupère une liste de films avec des options de filtrage par début, limite, et ordre. --> La fonction movies_list gère les paramètres de requête optionnels start, limit, et order : - start : Définit l'index de départ pour la liste des films (par défaut 0). - limit : Limite le nombre de films retournés (par défaut 100). - order : Trie les films soit par year (année) ou name (nom), si spécifié. Les champs d'ordre invalides renvoient une erreur 400. Elle récupère les films, applique le tri (si applicable) et retourne la liste filtrée des films au format JSON.

- `/actors/<id:int>` - Récupère un acteur par ID. --> La fonction actor_by_id prend un ID d'acteur, appelle get_actor(id) pour obtenir les détails de l'acteur, et retourne les informations de l'acteur au format JSON. Si l'acteur n'est pas trouvé, une erreur 404 est renvoyée avec le message "Acteur non trouvé". En cas d'erreur lors de la récupération, une erreur 404 est également renvoyée.

- `/actors` - Récupère une liste d'acteurs avec tri et pagination optionnels. --> La fonction actors_list gère les paramètres de requête start, limit, et order : - start : Définit l'index de départ pour la liste des acteurs (par défaut 0). - limit : Limite le nombre d'acteurs retournés (par défaut 100). - order : Spécifie l'ordre des acteurs en fonction de leur nom ou index. Si un tri est demandé, la liste des acteurs est triée en conséquence ; si des critères d'ordre invalides sont fournis, une erreur 400 est renvoyée. Elle retourne la liste d'acteurs triée et paginée au format JSON.

- `/actors/<id:int>/costars` - Récupère une liste des co-stars d'un acteur. --> La fonction actor_costars reçoit un ID d'acteur, vérifie si l'acteur existe et, dans ce cas, récupère tous les films dans lesquels il a joué. Elle trouve ensuite les autres acteurs ayant joué dans ces mêmes films (co-stars) et les retourne en tant que liste au format JSON. Si l'acteur n'existe pas, une erreur 404 est renvoyée avec le message "Acteur non trouvé".

- `/search/actors/<searchString>` - Recherche des acteurs par chaîne de recherche. --> La fonction search_actors appelle search_actor(searchString) pour trouver tous les acteurs correspondant à la chaîne de recherche. Elle retourne ensuite les 100 premiers acteurs correspondants au format JSON.

- `/search/movies/<searchString>` - Recherche des films par chaîne de recherche, avec filtrage optionnel. --> La fonction search_movies permet de rechercher des films par nom, avec un paramètre de requête filter optionnel : - filter : Une liste de paires clé-valeur séparées par des virgules pour filtrer, comme year:1964. Elle filtre les films correspondants selon les critères de filtrage fournis (par exemple, année). Les résultats sont retournés sous forme de tableau JSON des films filtrés. Les 100 premiers films correspondants sont retournés.

Gestion des Erreurs

- Erreur 404 : Si la ressource demandée n'est pas trouvée, l'API renvoie une réponse JSON avec le message d'erreur "Non trouvé".
- Erreur 400 : En cas de demande incorrecte, comme des paramètres invalides, l'API renvoie une réponse JSON avec le message d'erreur 

<span style="color:orange"><b>NOTE :</b></span> Nous avons fait appel aux fonctions get_movie, search_actor ...etc qui figurent dans le fichier utils.py via : <span style="color:gray"><b>from utils import movies, actors ,get_movie,get_actor,search_actor,search_movie :</b></span>."Mauvaise demande"."Mauvaise demande".

In [92]:
%%writefile app_movies_actors.py

from gevent import monkey
monkey.patch_all()
from bottle import Bottle, request, response, run
from bottle import route, run, response  
from utils import movies, actors ,get_movie,get_actor,search_actor,search_movie
import json 
import gzip
import threading
from bottle import route, run, response  

app = Bottle()

def error_response(status, message):
    response.status = status
    return json.dumps({"error": message})

# ***************************************************************
@app.route('/movies/<id>', methods=['GET'])
def movie_by_id(id):
    try: 
        id = int(id)
    except ValueError:
        return error_response(400, "Invalid movie ID")  

    if id < 0 or id >= len(movies):
        return error_response(400, "Invalid movie ID")  
    movie = get_movie(id)
    if not movie:
        return error_response(404, "Movie not found")  
    return movie

#*******************************************************************
@app.route('/movies', method='GET')
def movies_list():
    start = 0  
    limit = 100  
    order = None 
    if 'start' in request.params:
        try:
            start = int(request.params['start'])
        except ValueError:
            return error_response(400, "Invalid 'start' parameter")

    if 'limit' in request.params:
        try:
            limit = int(request.params['limit'])
        except ValueError:
            return error_response(400, "Invalid 'limit' parameter")

    if 'order' in request.params:
        order = request.params['order']
        
    sorted_movies = movies
    if order:
        if order == 'year':
            sorted_movies = sorted(movies, key=lambda m: m[2])  
        elif order == 'name':
            sorted_movies = sorted(movies, key=lambda m: m[0])  
        else:
            return error_response(400, "Invalid order field")

    selected_movies = sorted_movies[start:start + limit]
    return json.dumps([{'name': m[0], 'year': m[2], 'index': i} for i, m in enumerate(selected_movies)])

# *************************************************************************************************************
@app.route('/actors/<id>', methods=['GET'])
def actor_by_id(id):
    try: 
        id = int(id)
    except ValueError:
        return error_response(400, "Invalid actor ID")  

    if id < 0 or id >= len(actors):
        return error_response(400, "Invalid actor ID")  
    actor = get_actor(id)
    if not actor:
        return error_response(404, "Actor not found")  
    return actor
    
# **********************************************************************************************************
@app.route('/actors')
def actors_list():
    start = 0
    limit = 100
    order = None  
    if 'start' in request.params:
        try:
            start = int(request.params['start'])
        except ValueError:
            return error_response(400, "Invalid 'start' parameter")

    if 'limit' in request.params:
        try:
            limit = int(request.params['limit'])
        except ValueError:
            return error_response(400, "Invalid 'limit' parameter")

    if 'order' in request.params:
        order = request.params['order']

    sorted_actors = actors
    if order:
        try:
            sorted_actors = sorted(actors, key=lambda a: a[0] if order == 'name' else a[1])
        except KeyError:
            return error_response(400, "Invalid order field")

    selected_actors = sorted_actors[start:start + limit]
    return json.dumps([{'name': a[0], 'index': i} for i, a in enumerate(selected_actors)])

# ************************************************************************************************
@app.route('/actors/<id:int>/costars')
def actor_costars(id):
    if id < 0 or id >= len(actors):
        return error_response(404, "Actor not found")
    
    actor_movies = actors[id][1]
    costars = set()

    for movie_id in actor_movies:
        movie = movies[movie_id]
        costars.update(movie[1])

    costars.discard(id)  
    costar_list = [{'name': actors[c][0], 'index': c} for c in costars]
    return json.dumps(costar_list)


# ****************************************************************************************************
@app.route('/search/actors/<searchString>')
def search_actors(searchString):
    matching_actors = search_actor(searchString)
    return json.dumps(matching_actors[:100])


# ***************************************************************************************************
@app.route('/search/movies/<searchString>')
def search_movies(searchString):
    filter_param = request.query.filter
    filter_criteria = {}

    if filter_param:
        filters = filter_param.split(',')
        for f in filters:
            key, value = f.split(':')
            filter_criteria[key] = value

    matching_movies = search_movie(searchString)
    filtered_movies = []

    for movie in matching_movies:
        add_movie = True
        for key, value in filter_criteria.items():
            if key == 'year' and str(movie['year']) != value:
                add_movie = False
                break
        if add_movie:
            filtered_movies.append(movie)

    return json.dumps(filtered_movies[:100])


@app.error(404)
def error404(error):
    return json.dumps({"error": "Not found", "message": str(error)})

@app.error(400)
def error400(error):
    return json.dumps({"error": "Bad request", "message": str(error)})


Overwriting app_movies_actors.py


<span style="color:blue"><b>Solution :</b></span>

Route d'accueil : --> Il s'agit de la page principale de l'application web. Lorsqu'elle est accédée via une requête GET, elle renvoie simplement le message "Page principale". C'est une route de type placeholder qui pourrait être développée ultérieurement pour fournir des informations générales sur l'API ou d'autres services.

In [93]:
%%writefile app_user.py
from bottle import Bottle, request, response, run
import json
from utils import  movies, actors, get_movie,get_actor,search_actor,search_movie

app = Bottle()

@app.route('/')
def home():
    return "Page principale"

Overwriting app_user.py


<span style="color:blue"><b>Solution :</b></span>

Configuration et Exécution de l'Application

- L'application principale Bottle (app) est créée.
- L'application app_movies_actors est montée sur le chemin racine ("/"), ce qui signifie que toutes les routes définies dans app_movies_actors seront accessibles depuis l'URL de base du serveur.
- L'application app_user est intégrée dans l'application principale, ce qui rend toutes les routes définies dans app_user disponibles dans l'application principale.
- Enfin, la méthode app.run() est appelée pour démarrer le serveur en utilisant le serveur gevent sur localhost au port 8080.

In [94]:
%%writefile runn.py

from bottle import Bottle
from gevent import monkey; monkey.patch_all()
from app_movies_actors import app as app_movies_actors
from app_user import app as app_user

app = Bottle()
app.mount("/", app_movies_actors)
app.merge(app_user)
app.run(server="gevent", host="localhost", port=8080)

Overwriting runn.py


In [95]:
!wt python runn.py

## Exercise 7. Test a Web API

Using `pytest`, write a program that checks that the API made in the previous exercise works as expected.

Your answer here.

<span style="color:blue"><b>Solution :</b></span>

<span style="color:green"><b>` Tests pour la route /movies/<id:int> ` 
Cette section teste la récupération d'un film spécifique par son identifiant (ID).
</b></span>

* test_movie_by_id_valid :
    - Vérifie qu'un ID de film valide retourne un statut 200 et inclut les clés 'name' et 'year' dans les données du film.
    - Assure que l'API retourne bien les informations attendues pour un film existant.

* test_movie_by_id_invalid_range :
    - Vérifie que l'API retourne un statut 400 pour des ID non valides (négatifs ou trop grands), indiquant une erreur d'ID.
    - Teste les limites de validité de l'ID pour identifier des erreurs d'entrée.

* test_movie_by_id_invalid_type :
    - Vérifie que l'utilisation d'un ID de type non numérique (comme une chaîne de caractères) retourne un statut 400,
      confirmant que seuls les ID numériques sont acceptés.


<span style="color:green"><b>` Tests pour la route /movies avec des paramètres`
Cette section teste l'affichage de la liste des films avec ou sans paramètres optionnels.
</b></span>


* test_movies_list_default :
    - Vérifie que la route retourne une liste par défaut de films avec une limite maximum de 100 films.
    - Assure que les valeurs par défaut de l'API sont correctement appliquées en l'absence de paramètres.

* test_movies_list_start_limit :
    - Vérifie que les paramètres 'start' et 'limit' sont pris en compte, permettant la pagination des résultats.
    - Teste si la limite spécifiée est correctement appliquée.

* test_movies_invalid_parameter :
    - Vérifie que les paramètres de pagination invalides (comme 'start' non numérique) retournent un statut 400.
    - Assure que l'API gère correctement les entrées incorrectes.

* test_movies_ordre_by_year :
    - Vérifie que la liste des films peut être triée par année, confirmant que le tri fonctionne sans erreur.

* test_movies_invalid_order :
    - Vérifie qu'un tri par un champ invalide retourne un statut 400, indiquant une gestion correcte des champs de tri.


<span style="color:green"><b>` Tests pour la route /actors/<id:int>`
Cette section teste la récupération d'un acteur spécifique par son ID.
</b></span>


* test_actor_by_id_valid :
    - Vérifie qu'un ID d'acteur valide retourne un statut 200 et inclut les clés 'name' et 'movies' dans les données de l'acteur.

* test_actor_by_id_invalid_range :
    - Vérifie qu'un ID d'acteur non valide (négatif ou trop grand) retourne un statut 400 et un message d'erreur.

* test_actor_by_id_invalid_type :
    - Vérifie qu'un ID d'acteur de type non numérique retourne un statut 400, confirmant que seuls les ID numériques sont acceptés.



<span style="color:green"><b>` Tests pour la route /actors avec des paramètres `
Cette section teste la récupération de la liste des acteurs avec ou sans paramètres optionnels.
</b></span>

* test_actors_list_default :
    - Vérifie que la liste des acteurs est limitée à 100 par défaut, testant ainsi les valeurs par défaut.

* test_actors_list_start_limit :
    - Vérifie que les paramètres 'start' et 'limit' contrôlent correctement la pagination.

* test_actors_invalid_limit :
    - Vérifie qu'un paramètre 'limit' non numérique retourne un statut 400, assurant la validation des paramètres.
 


<span style="color:green"><b> ` Tests pour la route /actors/<id>/costars `
Cette section teste la récupération de la liste des co-stars d'un acteur donné.
</b></span>

* test_actors_costars_valid_ID :
    - Vérifie qu'un ID d'acteur valide retourne une liste de co-stars sous forme de tableau.

* test_actors_costars_invalid_ID :
    - Vérifie qu'un ID d'acteur inexistant retourne un statut 404, confirmant le bon traitement des erreurs d'ID.


<span style="color:green"><b> ` Tests pour la route /search/actors/<searchString> `
Cette section teste la recherche d'acteurs en fonction d'une chaîne de recherche.

</b></span>

* test_search_actors_valid :
    - Vérifie qu'une recherche par nom retourne des acteurs dont les noms contiennent la chaîne donnée.

* test_search_actors_empty_result :
    - Vérifie qu'une recherche sans résultats correspondants retourne une liste vide, assurant la validité de la recherche.


<span style="color:green"><b> `  Tests pour la route /search/movies/<searchString>`
Cette section teste la recherche de films en fonction d'une chaîne de recherche et de filtres.

</b></span>

* test_search_movies_valid :
    - Vérifie qu'une recherche par titre de film retourne des films contenant la chaîne de recherche.

* test_search_movies_search_by_year :
    - Vérifie que le filtre par année retourne uniquement les films de l'année spécifiée.

* test_search_movies_empty_result :
    - Vérifie qu'une recherche sans résultats retourne une liste vide, validant la gestion des recherches infructueuses.

Ressources : On a repris les exemple de cours mais on a du utiliser chatgpt aussi pour gérer quelques erreurs notamment celle du test de la limite par exemple.

In [99]:
%%writefile test_app.py
from requests import get
from json import loads
import threading
import time
import requests
import pytest
# from runn import run_server
server_ip = "localhost"
server_port = 8080


# ---------- Test sur /movies/<id:int> route
def test_movie_by_id_valid():
    r1 = get(f"http://{server_ip}:{server_port}/movies/55030")
    assert r1.status_code == 200
    movie_data = loads(r1.text)
    assert 'name' in movie_data
    assert 'year' in movie_data

def test_movie_by_id_invalid_range():
    r2 = get(f"http://{server_ip}:{server_port}/movies/-1")
    assert r2.status_code == 400  
    error = loads(r2.text)
    assert error["error"] == "Invalid movie ID" 
    
    r3 = get(f"http://{server_ip}:{server_port}/movies/444554545454545")
    assert r3.status_code == 400  
    error = loads(r3.text)
    assert error["error"] == "Invalid movie ID" 

def test_movie_by_id_invalid_type():
    r4 = get(f"http://{server_ip}:{server_port}/movies/hieuhfiehro")
    assert r4.status_code == 400  
    error = loads(r4.text)
    assert error["error"] == "Invalid movie ID"  

# ------------------------------ Test sur /movies route avec des paramètres différents
def test_movies_list_default():
    r1 = get(f"http://{server_ip}:{server_port}/movies")
    assert r1.status_code == 200
    movies_data = loads(r1.text)
    assert len(movies_data) <= 100 

def test_movies_list_start_limit():
    r2 = get(f"http://{server_ip}:{server_port}/movies?start=10&limit=5")
    assert r2.status_code == 200
    movies_data = loads(r2.text)
    assert len(movies_data) <= 5

def test_movies_invalid_parameter():
    r3 = get(f"http://{server_ip}:{server_port}/movies?start=invalid")
    assert r3.status_code == 400
    error = loads(r3.text)
    assert error["error"] == "Invalid 'start' parameter"

def test_movies_ordre_by_year():
    r4 = get(f"http://{server_ip}:{server_port}/movies?order=year")
    assert r4.status_code == 200

def test_movies_invalid_order():
    r5 = get(f"http://{server_ip}:{server_port}/movies?order=invalid")
    assert r5.status_code == 400
    error = loads(r5.text)
    assert error["error"] == "Invalid order field"

# ****************************************** Test for /actors/<id:int> route
def test_actor_by_id_valid():
    r1 = get(f"http://{server_ip}:{server_port}/actors/548029")
    assert r1.status_code == 200
    actor_data = loads(r1.text)
    assert 'name' in actor_data
    assert 'movies' in actor_data

def test_actor_by_id_invalid_range():
    r2 = get(f"http://{server_ip}:{server_port}/actors/-5")
    assert r2.status_code == 400
    error = loads(r2.text)
    assert error["error"] == "Invalid actor ID"

    r3 = get(f"http://{server_ip}:{server_port}/actors/444554545454545")
    assert r3.status_code == 400  
    error = loads(r3.text)
    assert error["error"] == "Invalid actor ID"  

def test_actor_by_id_invalid_type():
    r4 = get(f"http://{server_ip}:{server_port}/actors/sidhishsfohi")
    assert r4.status_code == 400  
    error = loads(r4.text)
    assert error["error"] == "Invalid actor ID"  

#*******************************Test pour /actors route avec différents paramètres
def test_actors_list_default():
    r1 = get(f"http://{server_ip}:{server_port}/actors")
    assert r1.status_code == 200
    actors_data = loads(r1.text)
    assert len(actors_data) <= 100 

def test_actors_list_start_limit():
    r2 = get(f"http://{server_ip}:{server_port}/actors?start=10&limit=5")
    assert r2.status_code == 200
    actors_data = loads(r2.text)
    assert len(actors_data) <= 5

def test_actors_invalid_limit():   
    r3 = get(f"http://{server_ip}:{server_port}/actors?limit=invalid")
    assert r3.status_code == 400
    error = loads(r3.text)
    assert error["error"] == "Invalid 'limit' parameter"

# ****************************** Test pour /actors/<id>/costars route
def test_actors_costars_valid_ID():
    r1 = get(f"http://{server_ip}:{server_port}/actors/548029/costars")
    assert r1.status_code == 200
    costars_data = loads(r1.text)
    assert isinstance(costars_data, list)

def test_actors_costars_invalid_ID():
    r2 = get(f"http://{server_ip}:{server_port}/actors/99999999/costars")
    assert r2.status_code == 404
    error = loads(r2.text)
    assert error["error"] == "Actor not found"

    

# ******************* Test pour /search/actors/<searchString> route
def test_search_actors_valid():
    r1 = get(f"http://{server_ip}:{server_port}/search/actors/Daniel")
    assert r1.status_code == 200
    actors_data = loads(r1.text)
    assert all('Daniel' in actor['name'] for actor in actors_data)

def test_search_actors_empty_result():
    r2 = get(f"http://{server_ip}:{server_port}/search/actors/NonexistentName")
    assert r2.status_code == 200
    actors_data = loads(r2.text)
    assert actors_data == []


# ******************** Test pour /search/movies/<searchString> route
def test_search_movies_valid():
    r1 = get(f"http://{server_ip}:{server_port}/search/movies/gendarme")
    assert r1.status_code == 200
    movies_data = loads(r1.text)
    assert all('gendarme' in movie['name'].lower() for movie in movies_data)


def test_search_movies_search_by_year():
    r2 = get(f"http://{server_ip}:{server_port}/search/movies/gendarme?filter=year:1964")
    assert r2.status_code == 200
    movies_data = loads(r2.text)
    assert all(movie['year'] == 1964 for movie in movies_data)

 
def test_search_movies_empty_result():
    r3 = get(f"http://{server_ip}:{server_port}/search/movies/NonexistentMovie")
    assert r3.status_code == 200
    movies_data = loads(r3.text)
    assert movies_data == []



Overwriting test_app.py


In [100]:
!pytest -v test_app.py -p no:warnings

============================= test session starts =============================
platform win32 -- Python 3.12.4, pytest-7.4.4, pluggy-1.0.0 -- D:\anaconda\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\user\OneDrive\Bureau
plugins: anyio-4.2.0
collecting ... collected 21 items

test_app.py::test_movie_by_id_valid PASSED                               [  4%]
test_app.py::test_movie_by_id_invalid_range PASSED                       [  9%]
test_app.py::test_movie_by_id_invalid_type PASSED                        [ 14%]
test_app.py::test_movies_list_default PASSED                             [ 19%]
test_app.py::test_movies_list_start_limit PASSED                         [ 23%]
test_app.py::test_movies_invalid_parameter PASSED                        [ 28%]
test_app.py::test_movies_ordre_by_year PASSED                            [ 33%]
test_app.py::test_movies_invalid_order PASSED                            [ 38%]
test_app.py::test_actor_by_id_valid PASSED                               [ 

## Exercise 8. Make a Website that uses the Web API

Create a Python web server using the Bottle library that utilizes the Web API you developed to offer the user a graphical Web interface. This interface allows the user to obtain, by entering relevant information into a Web form:

- The complete list of movies and the complete list of costars of an actor, possibly sorted alphabetically. This actor can be searched beforehand using a substring of characters appearing in her name.
- The colloration distance between two actors. As above, the actors can be searched beforehand using a substring of characters appearing in their names. Try to format a bit (not too much). For example:
  - The collaboration distance between Kevin Bacon and Jean Dujardin is 2.
  - Kevin bacon played in Wild things with Bill Murray;
  - Bill Murray played in The Monuments Men with Jean Dujardin.

Your answer here.

<span style="color:blue"><b>Solution :</b></span>

- Nous avons demandé à chatgpt de nous générer ce template.

In [104]:
%%writefile form.tpl
<html>
<head><title>Movie & Actor Finder</title></head>
<body>
    <h1>Recherche de Films et Acteurs</h1>
    <form action="/" method="post">
        <label>Titre du film :</label>
        <input type="text" name="title">
        <input type="hidden" name="action" value="movie">
        <button type="submit">Rechercher Film</button>
    </form>
    <br>
    <form action="/" method="post">
        <label>Nom de l'acteur :</label>
        <input type="text" name="name">
        <input type="hidden" name="action" value="actor">
        <button type="submit">Rechercher Acteur</button>
    </form>
</body>
</html>


Overwriting form.tpl


In [105]:
%%writefile result.tpl
<html>
<head><title>Résultats</title></head>
<body>
    <h1>Résultats</h1>
    % if result:
        <ul>
        % for item in result:
            <li>{{item}}</li>
        % end
        </ul>
    % else:
        <p>Aucun résultat trouvé.</p>
    % end
    <a href="/">Retour</a>
</body>
</html>


Overwriting result.tpl


<span style="color:blue"><b>Solution :</b></span>

- @app.get("/")
    - Affiche un formulaire permettant à l'utilisateur de choisir entre une recherche de film ou d'acteur.
    - SORTIE : Retourne un template HTML 'form.tpl' contenant le formulaire pour l'entrée utilisateur.
    
- @app.post("/")
    - Traite les données du formulaire pour effectuer une recherche en fonction de la sélection de l'utilisateur.
    - ENTRÉE (données du formulaire) : 
        - `action` : Type de recherche ('movie' pour film ou 'actor' pour acteur).
        - `title` (si action == "movie") : Titre du film à rechercher.
        - `name` (si action == "actor") : Nom de l'acteur à rechercher.
    - SORTIE : Retourne les résultats de la recherche dans un template 'result.tpl'.
    - ACTIONS :
        - Pour une recherche de film, envoie une requête à l'API pour rechercher un titre de film.
        - Pour une recherche d'acteur, envoie une requête à l'API pour rechercher un nom d'acteur.

In [106]:
%%writefile app_user2.py

from bottle import Bottle, request, template, run
import requests
from json import loads

app = Bottle()

@app.get("/")
def input_form():
    return template("form.tpl")  # On utilisera un template pour les formulaires

@app.post("/")
def process_form():
    action = request.forms.get("action")
    result = ""
    
    if action == "movie":
        title = request.forms.get("title")
        r = requests.get(f"http://localhost:8080/search/movies/{title}")
        result = loads(r.text)
    elif action == "actor":
        name = request.forms.get("name")
        r = requests.get(f"http://localhost:8080/search/actors/{name}")
        result = loads(r.text)
        
    return template("result.tpl", result=result)


Overwriting app_user2.py


<span style="color:blue"><b>Solution :</b></span>

Configuration et Exécution de l'Application

- L'application principale Bottle (app) est créée.
- L'application app_movies_actors est montée sur le chemin racine ("/"), ce qui signifie que toutes les routes définies dans app_movies_actors seront accessibles depuis l'URL de base du serveur.
- L'application app_user est intégrée dans l'application principale, ce qui rend toutes les routes définies dans app_user2 disponibles dans l'application principale.
- Enfin, la méthode app.run() est appelée pour démarrer le serveur en utilisant le serveur gevent sur localhost au port 8081.

In [163]:
%%writefile runn2.py
from bottle import Bottle
from gevent import monkey; monkey.patch_all()
from app_movies_actors import app as app_movies_actors
from app_user2 import app as app_user2

app = Bottle()
app.mount("/", app_movies_actors)
app.merge(app_user2)


app.run(server="gevent", host="localhost", port=8081)


Overwriting runn2.py


In [108]:
!wt python runn2.py